# Redes Sociais - APS 1


Uma rede de inclusão de arquivo de código-fonte Linux (v3.16). Os vértices representam arquivos fonte e uma aresta direcionada indica se um arquivo inclui outro.

_Nota: os grafos são dirigidos, conforme informado no agregador de databases [Netzschleuder](https://networks.skewed.de/), do qual foi extraído a base utilizada aqui._

## Pré-requisitos

In [1]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw

In [4]:
PATH = 'edges.csv'

## Análise dos dados importados

O arquivo relevante para nossa análise é `edges.csv`, que representa uma lista de valores separados por _vírgulas_, com uma aresta por linha. O significado de cada coluna do arquivo é:

- Primeira coluna: ID do nó de origem (_source_)

- Segunda coluna: ID do nó de destino (_target_)

Agora, devemos inspecionar os valores armazenados em `edges.csv`:

In [5]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Imprime o nó A que referencia e o nó B que é referenciado.
        print(parts[0], parts[1])

        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

0 9

0 10

0 11

0 12

0 13



## Criação do grafo

Utilizaremos a biblioteca [graph-tool](https://graph-tool.skewed.de/) somente para criação e visualização básica dos grafos, sem suporte de nenhum método ou função que não seja essencial.

In [6]:
g = gte.Graph(directed=True) # pois o grafo é directed, como informado acima.

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [7]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

def get_or_add_edge(g, id_origin, id_destiny):
    e = g.edge_by_ids(id_origin, id_destiny)
    if e is None:
        e = g.add_edge_by_ids(id_origin, id_destiny)
    return e

Depois de criar o novo grafo, vamos armazenar os valores de `out.linux` nele, o transformando em uma rede.

In [8]:
with open(PATH) as file:

    # Cria index de contagem de linhas lidas.
    i = 1
    
    # Ignora o cabeçalho.
    next(file)

    # Itera linha a linha do arquivo `out.linux`
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Define os IDs de origem e destino.
        id_origin = int(parts[0])
        id_destiny = int(parts[1])
        
        # Adiciona os vértices.
        get_or_add_vertex(g, id_origin)
        get_or_add_vertex(g, id_destiny)

        # Adiciona a aresta correspondente a esta linha.
        get_or_add_edge(g, id_origin, id_destiny)
        
        # Incrementa o contador de linhas lidas.
        i += 1

# Imprime a quantidade de linhas lidas.
print(f'Foram lidas {i} linhas.') 

Foram lidas 213955 linhas.


A seguir, devemos chamar `draw.sfdp_layout`, passando a rede, para rodar um algoritmo de posicionamento chamado SFDP [[1](#sfdp)].

Esse algoritmo usa uma ideia conhecida como [force-directed graph drawing](https://en.wikipedia.org/wiki/Force-directed_graph_drawing) para posicionar os vértices de forma a evidenciar agrupamentos.

In [9]:
layout = draw.sfdp_layout(g)

In [10]:
gte.move(g, layout)

## Armazenamento da rede

Para garantir a segurança da informação processada, devemos guardá-la em um arquivo na mesma pasta deste notebook.

In [16]:
g = gte.clean(g)
gte.save(g, 'linux.net.gz')

## Visualização da rede

O próximo passo é a renderização da rede.

In [17]:
r = netpixi.render('linux.net.gz', infinite = True)

Por fim, devemos ajustar a visualização da renderização.

In [18]:
r.vertex_default(size=4, bwidth=1)

In [19]:
r.edge_default(width=1)